In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np
import datetime
import time

##Random Sampling of data without replacement

In [0]:
filenames = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']

custom_df = pd.DataFrame()

for name in filenames:
  path = "/content/drive/My Drive/PROJECT 1001/Data4/fin_" + name + ".csv"
  df = pd.read_csv(path)
  df = df[['YEAR', 'QUARTER', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'FL_DATE',
           'OP_UNIQUE_CARRIER', 'ORIGIN', 'DEST', 'DEP_DELAY', 'DEP_DEL15', 'DEP_DELAY_GROUP',
           'ARR_DELAY', 'ARR_DEL15', 'ARR_DELAY_GROUP', 'AIR_TIME', 'DISTANCE', 'SCHED_DEP_TIME']]
  df = df[df['ORIGIN']!='AZA'] # dropping AZA since there are too few rows for sampling
  size = 610
  replace=False
  fn = lambda obj: obj.loc[np.random.choice(obj.index, size, replace),:]
  df['UNIQUE'] = list(zip(df.ORIGIN, df.MONTH, df.ARR_DEL15))
  df = df.groupby('UNIQUE', as_index=False).apply(fn)
  df['AIRPORT_DATE'] = list(zip(df.ORIGIN, df.FL_DATE))
  df['ORIGIN_COMBINED'] = list(zip(df.ORIGIN, df.MONTH))
  df['DEST_COMBINED'] = list(zip(df.DEST, df.MONTH))
  custom_df = custom_df.append(df)

In [0]:
# Length of dataset = months * delay/no-delay * number of airports * sample_size = 12*2*24*610 = 351360
len(custom_df)

351360

##JOINING WITH THE FEATURE LOOKUP TABLES

In [0]:
custom_df['AIRPORT_DATE'] = custom_df['AIRPORT_DATE'].astype(str)
custom_df['ORIGIN_COMBINED'] = custom_df['ORIGIN_COMBINED'].astype(str)
custom_df['DEST_COMBINED'] = custom_df['DEST_COMBINED'].astype(str)

Weather Data 

In [0]:
path = "/content/drive/My Drive/PROJECT 1001/weather_lookup.csv"
df_weather = pd.read_csv(path)
# Select PRCP (Precipitation), WDSP (Wind Speed), VISIB (Visibility), SLP (Sea Level Pressure) and AIRPORT_DATE for joining/ merging
df_weather = df_weather[['AIRPORT_DATE', 'PRCP', 'WDSP', 'VISIB', 'SLP']]
df_merge = custom_df.merge(df_weather, on='AIRPORT_DATE', how='left')
len(df_merge)

351360

Origin and Dest Data

In [0]:
path = "/content/drive/My Drive/PROJECT 1001/origin_delay_lookup.csv"
df_origin = pd.read_csv(path)
df_origin = df_origin.drop_duplicates()
df_merge = df_merge.merge(df_origin, on='ORIGIN_COMBINED', how='left')
len(df_merge)

351360

In [0]:
path = "/content/drive/My Drive/PROJECT 1001/dest_delay_lookup.csv"
df_dest = pd.read_csv(path)
df_dest['DEST_COMBINED'] = list(zip(df_dest['Unnamed: 0'], df_dest['Unnamed: 1']))
cols = [0,1]
df_dest.drop(df_dest.columns[cols], axis=1, inplace=True)
df_dest.rename(columns = {'DELAY_YES':'DELAY_YES_DEST', 'DELAY_NO':'DELAY_NO_DEST', 'TOTAL':'TOTAL_DEST', 'DELAY_TO_NO_DELAY_RATIO':'DELAY_NO_DELAY_RATIO_DEST',
                          'DELAY_PROB':'DELAY_PROB_DEST', 'DelayScore':'DelayScore_DEST'}, inplace = True)
df_dest['DEST_COMBINED'] = df_dest['DEST_COMBINED'].astype(str)
df_dest = df_dest.drop_duplicates()
df_merge = df_merge.merge(df_dest, on='DEST_COMBINED', how='left')
len(df_merge)

351360

Time based Features

In [0]:
df_merge['SCHED_DEP_TIME'] = df_merge['SCHED_DEP_TIME'].apply(pd.to_timedelta)
#type(df_merge['SCHED_DEP_TIME'][0])
timelist = ['00:00:00', '03:00:00', '06:00:00', '09:00:00', '12:00:00', '15:00:00', '18:00:00', '21:00:00', '24:00:00']
time_to_category = {}
bins = []
for i in range(len(timelist)):
  bins.append(pd.Timedelta(timelist[i]))
  time_to_category[timelist[i]] = i
print(time_to_category)
df_merge['TIME_GROUP'] = pd.cut(df_merge['SCHED_DEP_TIME'], bins, labels=timelist[:8])
df_merge['TIME_GROUP_NUM'] = df_merge['TIME_GROUP'].map(time_to_category)

{'00:00:00': 0, '03:00:00': 1, '06:00:00': 2, '09:00:00': 3, '12:00:00': 4, '15:00:00': 5, '18:00:00': 6, '21:00:00': 7, '24:00:00': 8}


In [0]:
path = "/content/drive/My Drive/PROJECT 1001/time_group_lookup.csv"
df_time = pd.read_csv(path)
df_time.rename(columns = {'ARR_DEL15':'TIME_ARR_DEL15', 'ARR_DELAY':'TIME_ARR_DELAY', 'DEP_DEL15':'TIME_DEP_DEL15', 'DEP_DELAY':'TIME_DEP_DELAY'}, inplace = True)
df_merge = df_merge.merge(df_time, on='TIME_GROUP', how='left')
len(df_merge)

351360

Carrier and Origin based delay features

In [0]:
path = "/content/drive/My Drive/PROJECT 1001/carrier_lookup.csv"
df_carrier = pd.read_csv(path)
df_carrier.rename(columns = {'TAXI_OUT':'CARRIER_TAXI_OUT', 'CARRIER_DELAY':'CARRIER_CARRIER_DELAY', 'NAS_DELAY':'CARRIER_NAS_DELAY'}, inplace = True)
df_merge = df_merge.merge(df_carrier, on='OP_UNIQUE_CARRIER', how='left')
len(df_merge)

351360

In [0]:
path = "/content/drive/My Drive/PROJECT 1001/origin_lookup.csv"
df_orig = pd.read_csv(path)
df_orig.rename(columns = {'TAXI_OUT':'ORIGIN_TAXI_OUT', 'CARRIER_DELAY':'ORIGIN_CARRIER_DELAY', 'NAS_DELAY':'ORIGIN_NAS_DELAY', 'LATE_AIRCRAFT_DELAY':'ORIGIN_LATE_AIRCRAFT_DELAY'}, inplace = True)
df_merge = df_merge.merge(df_orig, on='ORIGIN', how='left')
len(df_merge)

351360

In [0]:
df_merge.columns

Index(['YEAR', 'QUARTER', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'FL_DATE',
       'OP_UNIQUE_CARRIER', 'ORIGIN', 'DEST', 'DEP_DELAY', 'DEP_DEL15',
       'DEP_DELAY_GROUP', 'ARR_DELAY', 'ARR_DEL15', 'ARR_DELAY_GROUP',
       'AIR_TIME', 'DISTANCE', 'SCHED_DEP_TIME', 'UNIQUE', 'AIRPORT_DATE',
       'ORIGIN_COMBINED', 'DEST_COMBINED', 'PRCP', 'WDSP', 'VISIB', 'SLP',
       'DELAY_YES', 'DELAY_NO', 'TOTAL', 'DELAY_TO_NO_DELAY_RATIO',
       'DELAY_PROB', 'DelayScore', 'DELAY_YES_DEST', 'DELAY_NO_DEST',
       'TOTAL_DEST', 'DELAY_NO_DELAY_RATIO_DEST', 'DELAY_PROB_DEST',
       'DelayScore_DEST', 'TIME_GROUP', 'TIME_GROUP_NUM', 'Unnamed: 0_x',
       'TIME_ARR_DEL15', 'TIME_ARR_DELAY', 'TIME_DEP_DEL15', 'TIME_DEP_DELAY',
       'Unnamed: 0_y', 'CARRIER_TAXI_OUT', 'CARRIER_CARRIER_DELAY',
       'CARRIER_NAS_DELAY', 'Unnamed: 0', 'ORIGIN_TAXI_OUT',
       'ORIGIN_CARRIER_DELAY', 'ORIGIN_NAS_DELAY',
       'ORIGIN_LATE_AIRCRAFT_DELAY'],
      dtype='object')

Drop unusable features

In [0]:
drop_list = ['YEAR', 'FL_DATE', 'OP_UNIQUE_CARRIER', 'ORIGIN', 'DEST', 'DEP_DELAY', 'DEP_DEL15', 'DEP_DELAY_GROUP', 'ARR_DELAY_GROUP', 'SCHED_DEP_TIME', 'UNIQUE', 'AIRPORT_DATE',
             'ORIGIN_COMBINED', 'DEST_COMBINED', 'DELAY_YES', 'DELAY_NO', 'TOTAL', 'DELAY_YES_DEST', 'DELAY_NO_DEST', 'TOTAL_DEST', 'TIME_GROUP', 'Unnamed: 0_x',
             'Unnamed: 0_y', 'Unnamed: 0']
df_merge = df_merge.drop(drop_list, axis = 1)
df_merge.head()

,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,ARR_DELAY,ARR_DEL15,AIR_TIME,DISTANCE,PRCP,WDSP,VISIB,SLP,DELAY_TO_NO_DELAY_RATIO,DELAY_PROB,DelayScore,DELAY_NO_DELAY_RATIO_DEST,DELAY_PROB_DEST,DelayScore_DEST,TIME_GROUP_NUM,TIME_ARR_DEL15,TIME_ARR_DELAY,TIME_DEP_DEL15,TIME_DEP_DELAY,CARRIER_TAXI_OUT,CARRIER_CARRIER_DELAY,CARRIER_NAS_DELAY,ORIGIN_TAXI_OUT,ORIGIN_CARRIER_DELAY,ORIGIN_NAS_DELAY,ORIGIN_LATE_AIRCRAFT_DELAY
0,1,1,30,2,-23.0,0.0,251.0,1919.0,0.02,11.6,10.0,1025.7,0.189537,0.159337,3,0.146626,0.127876,2,3,0.166133,1.575064,0.150402,6.714174,18.168675,18.388806,17.685694,17.055532,18.620341,13.052774,21.548829
1,1,1,27,6,-22.0,0.0,107.0,765.0,0.00,7.0,10.0,1030.8,0.189537,0.159337,3,0.189076,0.159011,3,6,0.273603,11.905879,0.278546,17.489085,18.168675,18.388806,17.685694,17.055532,18.620341,13.052774,21.548829
2,1,1,22,1,-7.0,0.0,79.0,547.0,0.00,5.5,8.4,1019.4,0.189537,0.159337,3,0.239775,0.193402,4,4,0.204976,5.457473,0.198385,10.194971,18.168675,18.388806,17.685694,17.055532,18.620341,13.052774,21.548829
3,1,1,8,1,1.0,0.0,192.0,1599.0,0.00,7.9,9.4,1026.6,0.189537,0.159337,3,0.148148,0.129032,2,3,0.166133,1.575064,0.150402,6.714174,18.168675,18.388806,17.685694,17.055532,18.620341,13.052774,21.548829
4,1,1,15,1,-1.0,0.0,50.0,241.0,0.00,4.8,10.0,1031.7,0.189537,0.159337,3,0.350877,0.259740,5,6,0.273603,11.905879,0.278546,17.489085,23.094947,21.991548,12.894312,17.055532,18.620341,13.052774,21.548829


Two columns have null values that can be replaced with zero

In [0]:
df_merge = df_merge.fillna(0)
df_merge.isnull().any()

QUARTER                       False
MONTH                         False
DAY_OF_MONTH                  False
DAY_OF_WEEK                   False
ARR_DELAY                     False
ARR_DEL15                     False
AIR_TIME                      False
DISTANCE                      False
PRCP                          False
WDSP                          False
VISIB                         False
SLP                           False
DELAY_TO_NO_DELAY_RATIO       False
DELAY_PROB                    False
DelayScore                    False
DELAY_NO_DELAY_RATIO_DEST     False
DELAY_PROB_DEST               False
DelayScore_DEST               False
TIME_GROUP_NUM                False
TIME_ARR_DEL15                False
TIME_ARR_DELAY                False
TIME_DEP_DEL15                False
TIME_DEP_DELAY                False
CARRIER_TAXI_OUT              False
CARRIER_CARRIER_DELAY         False
CARRIER_NAS_DELAY             False
ORIGIN_TAXI_OUT               False
ORIGIN_CARRIER_DELAY        

Save final processed dataset

In [0]:
df_merge.to_csv('processed_dataset.csv')
!cp 'processed_dataset.csv' 'drive/My Drive/PROJECT 1001/'